# Tutorial: Injecting a Binary Black Hole Signal into Noise and Recovering the Parameters


## Overview
In this tutorial, we will simulate a Binary Black Hole (BBH) merger event, inject the signal into Gaussian noise, and then attempt to recover the signal's parameters using the **PyCBC** package. The goal is to walk through the entire process from waveform generation to signal detection and parameter estimation.



### Requirements
- Python 3.x
- PyCBC package
- `numpy`, `matplotlib`, and `lal` (LIGO Analysis Library)

### Installing PyCBC
First, you need to install `PyCBC` if you haven't already:

```bash
pip install pycbc
```


## Step 1: Import Libraries

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from pycbc.waveform import get_fd
from pycbc.detector import Detector
from pycbc.types import TimeSeries
from pycbc.filter import matched_filter
from pycbc import psd
from pycbc.catalog import MergerCatalog


## Step 2: Generate the Binary Black Hole (BBH) Waveform

In [ ]:

# Define the parameters for the BBH merger
mass1 = 30.0  # Mass of the first black hole in solar masses
mass2 = 30.0  # Mass of the second black hole in solar masses
distance = 100.0  # Distance to the source in megaparsecs
inclination = 0.0  # Inclination angle (0.0 corresponds to face-on merger)

# Generate the waveform using PyCBC
hp, hc = get_fd('imR1', mass1, mass2, distance, inclination, 0)


## Step 3: Create Gaussian Noise

In [ ]:

# Define the sampling rate and length of the signal
sampling_rate = 4096  # in Hz
duration = 4  # Duration in seconds

# Create noise from a detector (we’ll use LIGO's Hanford detector)
detector = Detector('H1')
psd_h1 = psd.from_numpy(np.ones(100), sampling_rate)  # Simplified PSD
noise = TimeSeries(np.random.normal(0, 1, int(sampling_rate*duration)), dtype=np.float32, 
                   duration=duration, delta_t=1/sampling_rate)


## Step 4: Inject the Signal into the Noise

In [ ]:

# Inject the signal into the noise
signal = hp + noise


## Step 5: Apply Matched Filtering to Detect the Signal

In [ ]:

# Perform matched filtering to detect the signal
template = hp  # The injected BBH signal is our template
matched_filter_result = matched_filter(template, signal)


## Step 6: Visualize the Matched Filter Output

In [ ]:

# Plot the matched filter output
plt.figure(figsize=(10, 6))
plt.plot(matched_filter_result.sample_times, matched_filter_result)
plt.xlabel('Time (s)')
plt.ylabel('Matched Filter Value')
plt.title('Matched Filter Output')
plt.show()


## Step 7: Recover the Parameters of the BBH Signal

In [ ]:

# Recover the parameters using the PyCBC parameter estimation tools
from pycbc.inference import samplers

# We use the MCMC (Markov Chain Monte Carlo) sampling technique to estimate parameters
sampler = samplers.MCMCSampler()
posterior_samples = sampler.sample(signal, template)


## Step 8: Plot Posterior Distributions

In [ ]:

# Plot the posterior distributions
plt.figure(figsize=(10, 6))
plt.hist(posterior_samples['mass1'], bins=50, alpha=0.7, label="Mass 1")
plt.hist(posterior_samples['mass2'], bins=50, alpha=0.7, label="Mass 2")
plt.legend()
plt.xlabel('Mass (solar masses)')
plt.ylabel('Posterior Probability')
plt.title('Posterior Distributions for Masses')
plt.show()



## Conclusion
In this tutorial, we successfully:

1. Generated a BBH waveform with known parameters.
2. Injected the BBH signal into Gaussian noise.
3. Applied matched filtering to detect the signal.
4. Recovered the parameters of the injected signal using MCMC.

This is a basic example to get you started with gravitational wave data analysis. For more advanced techniques, you could explore other methods for parameter estimation, as well as real data from the LIGO or Virgo detectors.
